In [1]:
import pandas as pd
import numpy as np
import pickle as pkl
import tensorflow as tf
import os, sys, math
import argparse
import config

tf.set_random_seed(seed=1991)
np.random.seed(seed=1991)

c:\users\timmy\anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
raw_df = pd.read_csv('./encoded_df.csv', encoding='cp949')

with open('./kv_map.pkl', 'rb') as f:
    kv_map = pkl.load(f)

In [3]:
config.col_dict
config.keep_prop = 0.7
config.train_size = 20480

In [4]:
train_df = raw_df[:config.train_size]
valid_df = raw_df[config.train_size:]

# split

In [5]:
train_df_cate = train_df.loc[:,['주야','요일', '발생지시도']]
train_df_cont = train_df.loc[:,['사망자수', '사상자수', '중상자수', '경상자수', '부상신고자수']]

In [6]:
valid_df_cate = valid_df.loc[:,['주야','요일', '발생지시도']]
valid_df_cont = valid_df.loc[:,['사망자수', '사상자수', '중상자수', '경상자수', '부상신고자수']]

In [7]:
np.random.seed(seed=910919)
config.vaild_drop_mask = np.random.rand(valid_df_cate.values.shape[0], valid_df_cate.values.shape[1])> (1- config.keep_prop)

# model

In [8]:
tf.reset_default_graph()
inp_cate = tf.placeholder(dtype=tf.int32, shape=[None,3], name='categorical_input_layer')
inp_cont = tf.placeholder(dtype=tf.int32, shape=[None,5], name='continuous_input_layer')
inp_cate_y = tf.placeholder(dtype=tf.int32, shape=[None,3], name='categorical_input_y_layer')
inp_cont_y = tf.placeholder(dtype=tf.int32, shape=[None,5], name='continuous_input_y_layer')

jy_oh = tf.one_hot(inp_cate[:, 0], depth=3)
yi_oh = tf.one_hot(inp_cate[:, 1], depth=8)
bj_oh = tf.one_hot(inp_cate[:, 2], depth=17)

jy_oh_y = tf.one_hot(inp_cate_y[:, 0], depth=3)
yi_oh_y = tf.one_hot(inp_cate_y[:, 1], depth=8)
bj_oh_y = tf.one_hot(inp_cate_y[:, 2], depth=17)


concat_cate = tf.concat([jy_oh, yi_oh, bj_oh], axis=-1)
concat_cate_y = tf.concat([jy_oh_y, yi_oh_y, bj_oh_y], axis=-1)


l1 = tf.contrib.layers.fully_connected(concat_cate, 10)

l2 = tf.contrib.layers.fully_connected(concat_cate, 28, activation_fn = tf.nn.sigmoid)

In [9]:
loss_op = tf.losses.mean_squared_error(concat_cate_y, l2)
optimizer = tf.train.AdamOptimizer()
train_op = optimizer.minimize(loss_op)

init = tf.global_variables_initializer()

In [10]:
config.batch_size = 1024

In [11]:
train_step = math.ceil(len(train_df)/ config.batch_size)
valid_step = math.ceil(len(valid_df)/ config.batch_size)

In [12]:
def run_session(sess, max_step, datas, config, epoch, mode=1):
    
    total_loss = 0.0
    if mode ==1:
        drop_mask = np.random.rand(datas[0].values.shape[0], datas[0].values.shape[1])> (1- config.keep_prop)
        droped_cate = datas[0].values * drop_mask
        
        for step in range(max_step):
            start_idx = step * config.batch_size
            loss_, _ = sess.run([loss_op, train_op], feed_dict = {inp_cate:droped_cate[start_idx:start_idx+config.batch_size], 
                                                                  inp_cont:datas[1].values[start_idx:start_idx+config.batch_size], 
                                                                  inp_cate_y:datas[0].values[start_idx:start_idx+config.batch_size]}
                               )
            
            total_loss += loss_
        total_loss /= max_step
        return total_loss 
    elif mode == 2:
        droped_valid_cate = datas[0].values * config.vaild_drop_mask
        
        for step in ange(max_step):
            start_idx = step * config.batch_size
            loss_, _ = sess.run([loss_op, train_op], feed_dict = {inp_cate:droped_valid_cate[start_idx:start_idx+config.batch_size], 
                                                                  inp_cont:datas[1].values[start_idx:start_idx+config.batch_size], 
                                                                  inp_cate_y:datas[0].values[start_idx:start_idx+config.batch_size]}
                               )
            
            total_loss += loss_
        total_loss /= max_step
        return total_loss 
    
    else:
        print('error')
        return total_loss

In [13]:
# =======================================================================================
# run session
with tf.Session() as sess:
    tf.set_random_seed(seed=1991)
    np.random.seed(seed=1991)
    sess.run(init)
    for epoch in range(5):
        print("Epoch : {}".format(epoch), end='\t')
        #train
        total_loss_ = run_session(sess, train_step, [train_df_cate, train_df_cont], config, epoch, mode=1)
        print("Train_loss : {}".format(total_loss_), end = '\t')
        
        total_loss_ = run_session(sess, train_step, [valid_df_cate, valid_df_cont], config, epoch, mode=1)
        print("Valid_loss : {}".format(total_loss_), end = '\n')

Epoch : 0	Train_loss : 0.24931342378258706	Valid_loss : 0.05989107266068459
Epoch : 1	Train_loss : 0.22725289165973664	Valid_loss : 0.05470441207289696
Epoch : 2	Train_loss : 0.20740632340312004	Valid_loss : 0.049957194924354555
Epoch : 3	Train_loss : 0.1892340712249279	Valid_loss : 0.04556216299533844
Epoch : 4	Train_loss : 0.1731152579188347	Valid_loss : 0.04178240299224854


In [ ]:
# test


In [14]:
valid_df_cate

,주야,요일,발생지시도
20480,2,8,14
20481,2,8,12
20482,2,8,14
20483,1,8,13
20484,1,8,16
20485,1,8,9
20486,1,8,13
20487,1,8,13
20488,1,8,15
20489,1,8,3


In [5]:
df

,주야,요일,사망자수,사상자수,중상자수,경상자수,부상신고자수,발생지시도
0,2,6,1,1,0,0,0,9
1,2,6,1,4,2,1,0,13
2,2,6,1,1,0,0,0,11
3,2,6,1,1,0,0,0,4
4,1,6,1,2,1,0,0,1
5,1,6,1,1,0,0,0,5
6,1,6,1,3,2,0,0,4
7,1,6,1,1,0,0,0,9
8,1,6,1,1,0,0,0,11
9,2,6,1,2,0,1,0,13
